<a href="https://colab.research.google.com/github/kmk4444/Retrieval-augmented-generation/blob/main/Part9_hyde.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Requirements.txt**

In [1]:
!touch requirements.txt
!echo langchain >> requirements.txt
!echo langchain-openai >> requirements.txt
!echo openai >> requirements.txt
!echo langchain-google-genai >> requirements.txt
!echo cohere >> requirements.txt
!echo faiss-cpu >> requirements.txt
!echo streamlit >> requirements.txt
!echo python-dotenv >> requirements.txt
!echo llama-index >> requirements.txt
!echo pypdf >> requirements.txt
!echo chromadb >> requirements.tx
!echo beautifulsoup4 >> requirements.tx
!echo matplotlib >> requirements.tx
!echo rank_bm25 >> requirements.tx
!echo replicate >> requirements.txt

**Bash/command**

In [ ]:
pip install -r requirements.txt

In [16]:
pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 50.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 80.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 7.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 4

In [17]:
%%writefile hydehelper.py
#https://g.co/gemini/share/f2219da5ecf4
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores.chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
import os
from dotenv import load_dotenv

load_dotenv()

#my_key_openai = os.getenv("openai_apikey")
#my_key_google = os.getenv("google_apikey")
my_key_openai="-"
my_key_google="--"

embeddings = OpenAIEmbeddings(api_key=my_key_openai)
#llm_gemini = ChatGoogleGenerativeAI(google_api_key=my_key_google, model = "gemini-pro") ERROR!
llm_openai = ChatOpenAI(api_key=my_key_openai, model = "gpt-4-0125-preview")

def load_and_split_documents(target_url):

  loader = WebBaseLoader(target_url)

  raw_documents = loader.load()

  text_splitter = RecursiveCharacterTextSplitter(
      chunk_size=1000,
      chunk_overlap=0,
      length_function=len
  )

  splitted_documents = text_splitter.split_documents(raw_documents)

  custom_documents = []

  for i, raw_doc in enumerate(splitted_documents):
    new_doc = Document(
        page_content = raw_doc.page_content,
        metadata = {
            "source": raw_doc.metadata["source"],
            "title" : raw_doc.metadata["title"],
            "description" : raw_doc.metadata["description"],
            "language" : raw_doc.metadata["language"],
            "doc_id" : i
        }
    )

    custom_documents.append(new_doc)

  return custom_documents

def get_relevant_documents(prompt, documents):

  vectorstore = Chroma.from_documents(documents, embeddings)

  retriever = vectorstore.as_retriever(search_type ="mmr")

  relevant_documents = retriever.get_relevant_documents(prompt)

  return relevant_documents

def run_rag(relevant_documents, prompt):
  context_data=""

  for document in relevant_documents:
    context_data = context_data + " " + document.page_content

  final_prompt = f"""Şöyle bir sorum var: {prompt}
  Bu soruyu yanıtlamak için elimizde şu bilgiler var: {context_data} .
  Bu sorunun yanıtını vermek için yalnızca sana burada verdiğim eldeki bilgileri kullan. Bunların dışına asla çıkma.
  """

  AI_Response = llm_openai.invoke(input=final_prompt)

  return AI_Response.content

def generate_hypothetical_document(prompt):

  HyDE_Prompt = f"""Kullanıcının sorusunu cevaplamak için kısa bir paragraf yaz.
  Kullanıcı Sorusu: {prompt}
  """

  hypothetical_answer = llm_openai.invoke(input=HyDE_Prompt)

  return hypothetical_answer.content

Overwriting hydehelper.py


In [12]:
%%writefile hyde.py
#https://gemini.google.com/app/4d6cadce75a9e057
import streamlit as st
import hydehelper

st.set_page_config(layout="wide")
st.title("Advanced RAG: Hyde | Kurgusal Yanıt Üzeriden Bellek Genişletme")
st.divider()

col_input, col_dummy1, col_hyde, col_dummy2, col_normal = st.columns([4,1,8,1,8])

with col_input:
    target_url = st.text_input(label="Hedef Web Adresini Giriniz", value="https://cbarkinozer.medium.com/reg%C3%BCle-edilmemi%C5%9F-yapay-zeka-teknolojileri-kullanman%C4%B1n-tehlikeleri-nelerdir-fa465da15491")
    original_prompt = st.text_input(label="Sorunuzu Giriniz:", value="Yapay zeka kullanımının yol açabileceği olumsuz durumlar nelerdir?")
    submit_btn = st.button(label="Gönder")
    st.divider()

with col_dummy1:
    st.empty()

with col_hyde:
    st.subheader("HyDE")
    st.empty()

with col_dummy2:
    st.empty()

with col_normal:
    st.subheader("Standart RAG")
    st.empty()


if submit_btn:

  splitted_documents = hydehelper.load_and_split_documents(target_url=target_url)

  HyDE_query = hydehelper.generate_hypothetical_document(prompt=original_prompt)

  col_input.subheader("Kurgusal Yanıt:")
  col_input.info(HyDE_query)

  relevant_documents = hydehelper.get_relevant_documents(prompt=HyDE_query, documents=splitted_documents)

  AI_Response = hydehelper.run_rag(relevant_documents=relevant_documents, prompt=original_prompt)

  col_hyde.info(AI_Response)
  col_hyde.divider()
  for rel_doc in relevant_documents:
      col_hyde.info(f"{rel_doc.metadata['doc_id']} || {rel_doc.page_content}")

  relevant_documents_normal = hydehelper.get_relevant_documents(prompt=original_prompt, documents=splitted_documents)

  AI_Response_normal = hydehelper.run_rag(relevant_documents=relevant_documents_normal, prompt=original_prompt)

  col_normal.success(AI_Response_normal)
  col_normal.divider()
  for rel_doc_n in relevant_documents_normal:
      col_normal.success(f"{rel_doc_n.metadata['doc_id']} || {rel_doc_n.page_content}")


Overwriting hyde.py


In [18]:
!npm install localtunnel
!streamlit run /content/hyde.py &>/content/logs.txt &
!npx localtunnel --port 8501

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
updated 1 package and audited 36 packages in 0.554s

3 packages are looking for funding
  run `npm fund` for details

found 2 moderate severity vulnerabilities
  run `npm audit fix` to fix them, or `npm audit` for details
npx: installed 22 in 2.306s
your url is: https://twelve-ghosts-fix.loca.lt
^C
